In [ ]:
# Load chat
import pandas as pd
import numpy as np
import re
import sqlalchemy
from whatstk import WhatsAppChat
from mysql.connector import connect
from sqlalchemy import create_engine
filepath = 'SE_chat.txt' # file created from WhatsApp group
chat = WhatsAppChat.from_source(filepath=filepath) #Use the WhatsAppChat library to convert group export to a dataframe
chat_df=chat.df
database_username = ''
database_password = ''
database_ip       = 'localhost'
database_name     = 'se_observation_log'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name)) # create the database connection
observable_object=pd.read_sql('SELECT * from observable_objects', database_connection) # table containing observable objects
members_name=pd.read_sql('SELECT * from members_name', database_connection) # table containing members_names



In [ ]:
chat_df['obv_date'] = pd.to_datetime(chat_df['date']).dt.date # convert the date column to date type
chat_df['obv_time'] = pd.to_datetime(chat_df['date']).dt.time # convert the time column to time type
chat_df=chat_df.drop(['date'],axis=1) # drops the date column
chat_df.tail()
chat_df['contains_observation']='' # creates new columns
chat_df['observation']=''
chat_df['obv_date'] = pd.to_datetime(chat_df['obv_date'], format='%Y-%m-%d') # Put date in readable format
observation='|'.join(observable_object['object']) # creates a list of items in the observable objects table in the database
chat_df['contains_observation']=chat_df.message.str.contains(observation, flags=re.IGNORECASE, regex=True)
    # checks the message column for items contained in the observation variable of the database.

In [ ]:
this_month=chat_df.loc[(chat_df['obv_date']>='2023-08-01') & (chat_df['obv_date']<='2023-08-31')] 
 # The observation is a monthly output. This creates a new dataframe for the specific month
this_month.to_csv('observation_log.csv') # outputs as CSV file
